# Beijing Air Quality

# Data Cleaning

In [69]:
df3 = pd.read_csv('PRSA_Data_20130301-20170228/PRSA_Data_Aotizhongxin_20130301-20170228.csv')

df3.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [1]:
# loading all needed packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import glob

%matplotlib inline

In [79]:

# From the DATA folder, we wil run through all of the individual csv files.
path = r'PRSA_Data_20130301-20170228/'
allFiles = glob.glob(path + '/*.csv')
# Prep the test and train data frames.
time = pd.DataFrame()
times = []
for file_ in allFiles:

    # Read in and set data index.
    df = pd.read_csv(file_,index_col = None,header = 0)
    df['Date'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.set_index('Date')
    df = df.loc[:'2016-12-31 23:00:00']
    df = df.drop(columns =['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN','wd', 'WSPM','No','year','month','day','hour'])
    df = df.resample('D').mean()


    # Interpolate the train data and add to the rest of the train list

    df = df.interpolate(method = 'time')
    times.append(df)

# Take the data frame we made at the top and combine the new data frame we processed here

time = pd.concat(times)
time.head()

,PM2.5
Date,
2013-03-01,8.625000
2013-03-02,36.916667
2013-03-03,80.333333
2013-03-04,26.500000
2013-03-05,136.708333


In [89]:
time = time.resample('D').mean()
time.shape

(1402, 1)

In [92]:
# From the DATA folder, we wil run through all of the individual csv files. 
path = r'PRSA_Data_20130301-20170228/'
allFiles = glob.glob(path + '/*.csv')
# Prep the test and train data frames.
test = pd.DataFrame()
train = pd.DataFrame()
trains = []
tests = []
for file_ in allFiles:
    
    # Read in and set data index. 
    df = pd.read_csv(file_,index_col = None,header = 0)
    df['Date'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.set_index('Date')
    #Limits to end 2016 
    df = df.loc[:'2016-12-31 23:00:00']
    df = df.drop(columns =['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN','wd', 'WSPM','No','year','month','day','hour'])

    df = df.resample('D').mean()
    
    # Drop unwanted columns, in univariate we should only have the date and the target. 
    
    # Train test split to prevent data leakage
    
    train = df[:int(df.shape[0]*0.8)]
    test = df[int(df.shape[0]*0.8):]
    
    
    # Interpolate the train data and add to the rest of the train list 
    
    train = train.interpolate(method = 'time')
    train['PM2.5'] = train['PM2.5'].fillna(method = 'bfill')
    trains.append(train)
    
    # Interpolate the test data and add to the rest of the test list
    
    test = test.interpolate(method = 'time')
    test['PM2.5'] = test['PM2.5'].fillna(method = 'bfill')
    tests.append(test)
    
    
# Take the data fame we made at the top and combine the new data frame we processed here 
    
train = pd.concat(trains)
test = pd.concat(tests)

# Resample to day mean 


In [83]:
train.index.value_counts()

2014-12-11    12
2014-08-19    12
2015-03-26    12
2015-11-03    12
2013-12-03    12
              ..
2013-08-05    12
2015-08-21    12
2014-01-10    12
2013-04-26    12
2013-05-21    12
Name: Date, Length: 1121, dtype: int64

In [85]:
test = test.resample('D').mean()

In [86]:
test.shape

(281, 1)

In [88]:
train = train.resample('D').mean()
train.shape

(1121, 1)

In [90]:
train.to_pickle('PKL/train.pkl')
test.to_pickle('PKL/test.pkl')
time.to_pickle('PKL/full.pkl')


In [77]:
time.head()

,PM2.5
Date,
2013-03-01,8.625000
2013-03-02,36.916667
2013-03-03,80.333333
2013-03-04,26.500000
2013-03-05,136.708333


In [78]:
train.head()

,PM2.5
Date,
2013-03-01,8.083333
2013-03-01,8.083333
2013-03-01,8.083333
2013-03-01,8.083333
2013-03-01,8.083333
